In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [305]:
model = """
t(0.5)::gender.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.
t(0.5)::smoke.
t(0.5)::alcohol.

t(0.33)::bmi_1;t(0.33)::bmi_2;t(0.33)::bmi_3.
t(0.5)::ldl.
t(0.33)::blood_pressure_1;t(0.33)::blood_pressure_2;t(0.33)::blood_pressure_3.

t(_)::blood_pressure_1:-gender.
t(_)::blood_pressure_1:-age_1.
t(_)::blood_pressure_1:-age_2.
t(_)::blood_pressure_1:-age_3.
t(_)::blood_pressure_1:-age_4.
t(_)::blood_pressure_1:-smoke.
t(_)::blood_pressure_1:-alcohol.

t(_)::blood_pressure_2:-gender.
t(_)::blood_pressure_2:-age_1.
t(_)::blood_pressure_2:-age_2.
t(_)::blood_pressure_2:-age_3.
t(_)::blood_pressure_2:-age_4.
t(_)::blood_pressure_2:-smoke.
t(_)::blood_pressure_2:-alcohol.

t(_)::blood_pressure_3:-gender.
t(_)::blood_pressure_3:-age_1.
t(_)::blood_pressure_3:-age_2.
t(_)::blood_pressure_3:-age_3.
t(_)::blood_pressure_3:-age_4.
t(_)::blood_pressure_3:-smoke.
t(_)::blood_pressure_3:-alcohol.

t(_)::ldl_t:-gender.
t(_)::ldl_t:-age_1.
t(_)::ldl_t:-age_2.
t(_)::ldl_t:-age_3.
t(_)::ldl_t:-age_4.
t(_)::ldl_t:-smoke.
t(_)::ldl_t:-alcohol.

t(_)::ldl_f:-gender.
t(_)::ldl_f:-age_1.
t(_)::ldl_f:-age_2.
t(_)::ldl_f:-age_3.
t(_)::ldl_f:-age_4.
t(_)::ldl_f:-smoke.
t(_)::ldl_f:-alcohol.

t(_)::bmi_1:-gender.
t(_)::bmi_1:-age_1.
t(_)::bmi_1:-age_2.
t(_)::bmi_1:-age_3.
t(_)::bmi_1:-age_4.
t(_)::bmi_1:-smoke.
t(_)::bmi_1:-alcohol.

t(_)::bmi_2:-gender.
t(_)::bmi_2:-age_1.
t(_)::bmi_2:-age_2.
t(_)::bmi_2:-age_3.
t(_)::bmi_2:-age_4.
t(_)::bmi_2:-smoke.
t(_)::bmi_2:-alcohol.

t(_)::bmi_3:-gender.
t(_)::bmi_3:-age_1.
t(_)::bmi_3:-age_2.
t(_)::bmi_3:-age_3.
t(_)::bmi_3:-age_4.
t(_)::bmi_3:-smoke.
t(_)::bmi_3:-alcohol.

t(_)::heart_d:-blood_pressure_1.
t(_)::heart_d:-blood_pressure_2.
t(_)::heart_d:-blood_pressure_3.
t(_)::heart_d:-ldl.
t(_)::heart_d:-bmi_1.
t(_)::heart_d:-bmi_2.
t(_)::heart_d:-bmi_3.

t(_)::glucose:-blood_pressure_1.
t(_)::glucose:-blood_pressure_2.
t(_)::glucose:-blood_pressure_3.
t(_)::glucose:-ldl.
t(_)::glucose:-bmi_1.
t(_)::glucose:-bmi_2.
t(_)::glucose:-bmi_3.
"""

In [306]:
gender = Term('gender')
age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')
smoke = Term('smoke')
alcohol = Term('alcohol')

bmi_1 = Term('bmi_1')
bmi_2 = Term('bmi_2')
bmi_3 = Term('bmi_3')
ldl = Term('ldl')
blood_pressure_1 = Term('blood_pressure_1')
blood_pressure_2 = Term('blood_pressure_2')
blood_pressure_3 = Term('blood_pressure_3')

heart_d = Term('heart_d')
glucose = Term('glucose')

In [319]:
examples = [
    [(gender, False), (age_1, True), (bmi_1, True)],
    [(gender, False), (age_2, True), (bmi_2, True)]
]

In [320]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), examples)

In [321]:
print (lfi_problem.get_model())

0.0::gender.
0.5::age_1; 0.5::age_2; 0.0::age_3; 0.0::age_4.
0.679812137809091::smoke.
0.933682118772443::alcohol.
0.999999401302505::bmi_1; 0.618480034879732::bmi_2; 0.33::bmi_3.
0.5::ldl.
0.33::blood_pressure_1; 0.33::blood_pressure_2; 0.33::blood_pressure_3.
0.066109930678042::blood_pressure_1 :- gender.
0.567804077549344::blood_pressure_1 :- age_1.
0.405937565571932::blood_pressure_1 :- age_2.
0.528850790702909::blood_pressure_1 :- age_3.
0.857489193031252::blood_pressure_1 :- age_4.
0.983811517556909::blood_pressure_1 :- smoke.
0.423787794845486::blood_pressure_1 :- alcohol.
0.666568176095751::blood_pressure_2 :- gender.
0.430253953213047::blood_pressure_2 :- age_1.
0.557471826381722::blood_pressure_2 :- age_2.
0.531311608826226::blood_pressure_2 :- age_3.
0.658844171835042::blood_pressure_2 :- age_4.
0.090124408537632::blood_pressure_2 :- smoke.
0.302344049475669::blood_pressure_2 :- alcohol.
0.564561111297846::blood_pressure_3 :- gender.
0.319684734144175::blood_pressure_3 :- ag